In [ ]:
import pandas as pd
import numpy as np

### Junção dos datasets

In [ ]:
wos = pd.read_excel('../../data/raw/wos_base.xlsx')
wos

In [ ]:
scopus = pd.read_csv('../../data/raw/scopus_base.csv')
scopus

In [ ]:
selecionados = pd.read_csv('../../data/processed/selection.csv')
selecionados

In [ ]:
selecionados_aceito = selecionados[selecionados['Status'] == 'Aceito'][['DOI', 'Abstract']]
selecionados_aceito['Abstract'] = selecionados_aceito['Abstract'].apply(lambda row: str(row).lower().strip())
selecionados_aceito['DOI'] = selecionados_aceito['DOI'].apply(lambda row: str(row).lower().strip())
selecionados_aceito

In [ ]:
dados = pd.concat([wos, scopus], ignore_index=True)
dados

In [ ]:
dados.columns

In [ ]:
dados[['AB', 'DI']] = dados[['AB', 'DI']].map(lambda row: str(row).lower().strip())
dados

In [ ]:
dados.drop_duplicates(inplace=True, subset=['DI'])
dados = dados[(dados['PY'] <= 2024) & (dados['PY'] >= 2014)]
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
selecionados_aceito.DOI.values

In [ ]:
indices_drop = []
for index, row in dados.iterrows():
  if (row.AB not in selecionados_aceito.Abstract.values) and (row.DI not in selecionados_aceito.DOI.values):
    indices_drop.append(index)

In [ ]:
len(indices_drop)

In [ ]:
dados.drop(indices_drop, inplace=True)
dados.drop_duplicates(inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados.to_excel('../../data/processed/filtereds_merged.xlsx', index=False)

### Tratamento artigos Wos + Scopus

A ideia aqui é padronizar o formato dos datasets e juntar os dois em um só. Permanecendo no formato do Scopus, que é mais completo.

In [ ]:
dados_merged = pd.read_excel('../../data/processed/filtereds_merged.xlsx')
dados_merged

In [ ]:
dados_merged.columns

In [ ]:
dados_merged.DI.values

In [ ]:
def split_pages(pp):
    try:
        start, end = pp.split('-')
        return int(start), int(end)
    except:
        return np.nan, np.nan

In [ ]:
df_scopus_mapped = pd.DataFrame({
    'Authors': dados_merged.get('AU', np.nan),
    'Author full names': dados_merged.get('AF', np.nan),
    'Author(s) ID': dados_merged.get('orcid.numbers', np.nan),
    'Title': dados_merged.get('TI', np.nan),
    'Year': dados_merged.get('PY', np.nan),
    'Source title': dados_merged.get('SO', np.nan),
    'Volume': dados_merged.get('VL', np.nan),
    'Issue': dados_merged.get('PN', np.nan),
    'Art. No.': dados_merged.get('BE', np.nan),
    'Page start': dados_merged.get('PP', '').apply(lambda x: split_pages(str(x))[0]),
    'Page end': dados_merged.get('PP', '').apply(lambda x: split_pages(str(x))[1]),
    'Page count': dados_merged.get('NR', np.nan),
    'Cited by': dados_merged.get('TC', np.nan),
    'DOI': dados_merged.get('DI', np.nan),
    'Link': dados_merged.get('url', np.nan),
    'Affiliations': dados_merged.get('C1', np.nan),
    'Authors with affiliations': dados_merged.get('RP', np.nan),
    'Abstract': dados_merged.get('AB', np.nan),
    'Author Keywords': dados_merged.get('DE', np.nan),
    'Index Keywords': dados_merged.get('ID', np.nan),
    'Molecular Sequence Numbers': np.nan,
    'Chemicals/CAS': np.nan,
    'Tradenames': np.nan,
    'Manufacturers': np.nan,
    'Funding Details': dados_merged.get('FU', np.nan),
    'Funding Texts': np.nan,
    'References': dados_merged.get('CR', np.nan),
    'Correspondence Address': dados_merged.get('C1', np.nan),
    'Editors': np.nan,
    'Publisher': dados_merged.get('PU', np.nan),
    'Sponsors': dados_merged.get('FU', np.nan),
    'Conference name': np.nan,
    'Conference date': np.nan,
    'Conference location': np.nan,
    'Conference code': np.nan,
    'ISSN': dados_merged.get('SN', np.nan),
    'ISBN': dados_merged.get('BN', np.nan),
    'CODEN': dados_merged.get('coden', np.nan),
    'PubMed ID': dados_merged.get('pmid', np.nan),
    'Language of Original Document': dados_merged.get('LA', np.nan),
    'Abbreviated Source Title': dados_merged.get('JI', np.nan),
    'Document Type': dados_merged.get('DT', np.nan),
    'Publication Stage': dados_merged.get('publication_stage', np.nan),
    'Open Access': dados_merged.get('oa', np.nan),
    'Source': dados_merged.get('DB', np.nan),
    'EID': dados_merged.get('UT', np.nan)
})

In [ ]:
print(df_scopus_mapped.head())

In [ ]:
df_scopus_mapped.to_excel("../../data/processed/merged_exported_scopus_mapped.xlsx", index=False)

### Seleção dos Wos a partir do Scopus

In [ ]:
dados_scopus_base = pd.read_csv("../../data/raw/scopus_base.csv")
dados_scopus_base

In [ ]:
dados_scopus_base.drop_duplicates(subset=['DOI'], inplace=True)
dados_scopus_base.reset_index(drop=True, inplace=True)
dados_scopus_base

In [ ]:
selecionados = pd.read_csv("../../data/processed/selection.csv")
selecionados

In [ ]:
selecionados = selecionados[selecionados['Status'] == 'Aceito']
selecionados

In [ ]:
selecionados = selecionados[['Título', 'Abstract', 'DOI', 'Fonte']]
selecionados

In [ ]:
selecionados_scopus = selecionados[selecionados['Fonte'] == 'Scopus']
selecionados_scopus.reset_index(drop=True, inplace=True)
selecionados_scopus

In [ ]:
selecionados_wos = selecionados[selecionados['Fonte'] == 'ISI']
selecionados_wos.reset_index(drop=True, inplace=True)
selecionados_wos

In [ ]:
incluidos = []
for index, row in dados_scopus_base.iterrows():
  for index_selecionado, row_selecionado in selecionados_scopus.iterrows():
    if (str(row.DOI).lower().strip() == str(row_selecionado.DOI).lower().strip()) and \
    (row.Title.lower().strip() == row_selecionado.Título.lower().strip()) and \
    (row.Abstract.lower().strip() == row_selecionado.Abstract.lower().strip()):
      incluidos.append(index)

In [ ]:
len(incluidos)

In [ ]:
scopus_filtrado = dados_scopus_base.loc[incluidos]
scopus_filtrado.reset_index(drop=True, inplace=True)
scopus_filtrado

In [ ]:
scopus_filtrado.to_csv("../../data/processed/scopus_filtered.csv", index=False)

In [ ]:
dados_wos_base = pd.read_excel("../../data/raw/wos_base.xls")
dados_wos_base

In [ ]:
dados_wos_base.columns

In [ ]:
incluidos_wos = []
for index, row in dados_wos_base.iterrows():
  for index_selecionado, row_selecionado in selecionados_wos.iterrows():
    if (str(row.DOI).lower().strip() == str(row_selecionado.DOI).lower().strip()) and \
    (row['Article Title'].lower().strip() == row_selecionado.Título.lower().strip()) and \
    (row.Abstract.lower().strip() == row_selecionado.Abstract.lower().strip()):
      incluidos_wos.append(index)

In [ ]:
len(incluidos_wos)

In [ ]:
wos_filtrado = dados_wos_base.loc[incluidos_wos]
wos_filtrado.drop_duplicates(subset=['DOI'], inplace=True)
wos_filtrado.reset_index(drop=True, inplace=True)
wos_filtrado

In [ ]:
wos_filtrado.to_csv("../../data/processed/wos_filtered.csv", index=False)

In [ ]:
scopus_filtrado.columns

In [ ]:
wos_filtrado.columns

In [ ]:
def mapping_databases(wos_register):
  return {
      "Authors": wos_register["Authors"],
      "Author full names": wos_register["Author Full Names"],
      "Author(s) ID": None,
      "Title": wos_register["Article Title"],
      "Year": wos_register["Publication Year"],
      "Source title": wos_register["Source Title"],
      "Volume": wos_register["Volume"],
      "Issue": wos_register["Issue"],
      "Art. No.": wos_register["Article Number"],
      "Page start": wos_register["Start Page"],
      "Page end": wos_register["End Page"],
      "Page count": wos_register["Number of Pages"],
      "Cited by": wos_register["Cited Reference Count"],
      "DOI": wos_register["DOI"],
      "Link": None,
      "Affiliations": wos_register["Affiliations"],
      "Authors with affiliations": None,
      "Abstract": wos_register["Abstract"],
      "Author Keywords": wos_register["Author Keywords"],
      "Index Keywords": wos_register["Keywords Plus"],
      "Molecular Sequence Numbers": None,
      "Chemicals/CAS" : None,
      "Tradenames" : None,
      "Manufacturers" : None,
      "Funding Details" : wos_register["Funding Orgs"],
      "Funding Texts" : wos_register["Funding Text"],
      "References" : wos_register["Cited References"],
      "Correspondence Address" : wos_register["Publisher Address"],
      "Editors" : wos_register["Book Editors"],
      "Publisher" : wos_register["Publisher"],
      "Sponsors" : wos_register["Conference Sponsor"],
      "Conference name" : wos_register["Conference Title"],
      "Conference date" : wos_register["Conference Date"],
      "Conference location" : wos_register["Conference Location"],
      "Conference code" : None,
      "ISSN" : wos_register["ISSN"],
      "ISBN" : wos_register["ISBN"],
      "CODEN" : None,
      "PubMed ID" : wos_register["Pubmed Id"],
      "Language of Original Document" : wos_register["Language"],
      "Abbreviated Source Title" : wos_register["Journal Abbreviation"],
      "Document Type" : wos_register["Document Type"],
      "Publication Stage" : None,
      "Open Access" : wos_register["Open Access Designations"],
      "Source" : "ISI",
      "EID" : wos_register["UT (Unique WOS ID)"],
  }

In [ ]:
wos_data = []
for indice in wos_filtrado.index:
  wos_data.append(mapping_databases(wos_filtrado.loc[indice]))

In [ ]:
wos_df = pd.DataFrame(wos_data)
wos_df

In [ ]:
wos_df.columns

In [ ]:
scopus_filtrado.columns

In [ ]:
dados_final = pd.concat([scopus_filtrado, wos_df])
dados_final.drop_duplicates(subset=['Abstract'], inplace=True)
dados_final.drop_duplicates(subset=['Title'], inplace=True)
dados_final.reset_index(drop=True, inplace=True)
dados_final

In [ ]:
dados_final.to_csv("../../data/ready/final_dataset_merged.csv", index=False)